In [ ]:
!pip install tensorflow_hub

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
train_data=pd.read_csv(r'C:\Users\My Pc\ML_project\text\Mental.txt', sep = ';')
train_data.head()

In [ ]:
def preprocess(data):
    data = pd.read_csv(r'C:\Users\My Pc\ML_project\text\Mental.txt', sep = ';')
    data.head()
    hos = []
    for i in range(len(data.emotion)):
        if data['emotion'][i] in ['joy', 'love', 'surprise']:
            hos.append(1) # happy is 1
        else:
            hos.append(0) # sad is 0
    data['hos'] = hos
    return data


In [ ]:
train_data = preprocess('train')
train = train_data.copy()

In [ ]:
train['emotion'].value_counts()

In [ ]:
train[train.emotion=='surpise']['text'].head(10)

In [ ]:
train.groupby('hos').describe()

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"


In [ ]:
hub_layer=hub.KerasLayer(model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
hub_layer(train['text'][:3])

In [ ]:
model=tf.keras.Sequential()


In [ ]:
model.add(hub_layer)


In [ ]:
model.add(tf.keras.layers.Dense(16, activation='relu'))

In [ ]:
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

In [ ]:
from tensorflow.python.keras.utils import losses_utils
from tensorflow.python.ops.losses import util as tf_losses_util



In [ ]:
model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(from_logits=True), 
             metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
val=preprocess('val')

In [ ]:
val.head()

In [ ]:
history=model.fit(train.text, train.hos, epochs=40, batch_size=512, validation_data=(val.text, val.hos), verbose=0)

In [ ]:
test=preprocess('test')

In [ ]:
results=model.evaluate(test['text'], test['hos'])

In [ ]:
print(results)

In [ ]:
preds=model.predict(test.text)

In [ ]:
predstr=model.predict(train.text)

In [ ]:
print(predstr.min(),predstr.max())

In [ ]:
print(preds.min(), preds.max())

In [ ]:
def postprocessor(preds):
    for i in preds:
        norm_preds.append((i - predstr.min()) / range)
        probab.append((i - predstr.min()) * 100 / range)
    return np.mean(probab)#, probab

#print(postprocessor(preds))

answers = []
answers.append(input('How would you describe your experience at your workplace/college/school in the past few days?'))
answers.append(input('How do you like to spend your leisure time? How do you feel after it?'))
answers.append(input('Life has its ups and downs. Although handling successes can be difficult, setbacks can affect mental health strongly. How do you manage your emotions after failures?'))
answers.append(input('Are there any improvements/decline in your salary/grades?'))
#answers.append(input('Any recent unpleasant experience that you would like to share?'))
answers.append(input('In a broad sense, how would you describe the way your life is going on?'))
#answers.append(input('How would you describe your experience at your workplace/college/school in the past few days?'))
results = model.predict(answers)
score = postprocessor(results)
print('Your mental health score is:', score)

if score < 25:
    print("You are going through a bad phase in life. But don't worry, bad times are not permanent. Try to seek help from a trained professional to improve your mental health.")
else:
    print("Your mental health looks great! Continue enjoying life and try to help others who are struggling with their mental health.")
    